Partially complete import of Ks files and output to stats.csv


In [0]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it


from sklearn.cluster import KMeans

bodies = pd.read_csv('correctedframes.csv')
chars = pd.read_csv('correctedchars.csv')
gliders = pd.read_csv('correctedgliders.csv')
tires = pd.read_csv('correctedtires.csv')

# use only stock (non-DLC) characters / karts / tires
#chars = chars.loc[chars['DLC']==0]
#bodies = bodies.loc[bodies['DLC']==0]
#tires = tires.loc[tires['DLC']==0]
#gliders = gliders.loc[gliders['DLC']==0]

stat_cols = bodies.columns[3:-1]
main_cols = ['Weight','Speed','Acceleration','Handling','Traction']

# lots of characters/karts/tires are exactly the same. here we just want one from each stat type
chars_unique = chars.drop_duplicates(subset=stat_cols).set_index('CBTG')[stat_cols]
bodies_unique = bodies.drop_duplicates(subset=stat_cols).set_index('CBTG')[stat_cols]
tires_unique = tires.drop_duplicates(subset=stat_cols).set_index('CBTG')[stat_cols]
gliders_unique = gliders.drop_duplicates(subset=stat_cols).set_index('CBTG')[stat_cols]





In [0]:
combos=[]

body_names=bodies_unique.index

tire_names=tires_unique.index

char_names=chars_unique.index

glider_names=gliders_unique.index

for body in body_names:
    for tire in tire_names:
        for char in char_names: 
            for glider in glider_names: 
                thiscombo=(char,body,tire,glider)
                combos.append(thiscombo)

stats=pd.DataFrame(columns=['speed','accel','hand'], index=combos)


for combo in combos:
    #print(combo)
    char=combo[0]
    body=combo[1]
    tire=combo[2]
    glider=combo[3]
    speed=sum([gliders_unique.loc[glider,'Speed'],bodies_unique.loc[body,'Speed'],tires_unique.loc[tire,'Speed'],chars_unique.loc[char,'Speed']] )
    accel= sum([gliders_unique.loc[glider,'Acceleration'],bodies_unique.loc[body,'Acceleration'],tires_unique.loc[tire,'Acceleration'],chars_unique.loc[char,'Acceleration'] ])
    hand= sum([gliders_unique.loc[glider,'Handling'],bodies_unique.loc[body,'Handling'],tires_unique.loc[tire,'Handling'],chars_unique.loc[char,'Handling'] ])
    
    index=combo
#    print(index)
    stats.loc[(index),'speed':'hand']= [speed, accel, hand]
#    stats.loc[(index),'accel']=accel
#   stats.loc[(index),'hand']=hand
#    print(speed, accel, hand)

In [0]:
stats.to_csv('stats.csv')

In [0]:
maxes=[max(stats.loc[:,'speed']),max(stats.loc[:,'accel']),max(stats.loc[:,'hand'])]

print(maxes)
def is_pareto_front(row, maxes):
    cols=len(row)
    for i in range(0,3):
        if row[i]==maxes[i]:
            return True
    return False



import numpy as np


def is_pareto_efficient_dumb(costs):
    """
    :param costs: An (n_points, n_costs) array
    :return: A (n_points, ) boolean array, indicating whether each point is Pareto efficient
    """
    is_efficient = np.ones(costs.shape[0], dtype = bool)
    for i, c in enumerate(costs):
        is_efficient[i] = np.all(np.any(costs>=c, axis=1))
    return is_efficient


pareto=pd.DataFrame()

for index,row in stats.iterrows():
#    print(index,row)
    if is_pareto_front(row, maxes):
        print(index,row)
        pareto[index]=row
        
pareto

In [0]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



fig = plt.figure()
ax = fig.add_subplot(221, projection='3d')

x = stats.loc[:,'speed']
y =stats.loc[:,'accel']
z =stats.loc[:,'hand']

ax.scatter(x, y, z, c='r', marker='o')

ax.set_xlabel('speed')
ax.set_ylabel('accel')
ax.set_zlabel('hand')

axy=fig.add_subplot(222)
axy.scatter(x, y)

ayz=fig.add_subplot(223)
ayz.scatter(y, z)

axz=fig.add_subplot(224)
axz.scatter(x, z)


plt.show()